# Habib's vectorbt
pip install all necessary libraries.
import all necessary libraries.
talib is the most technical to install.
use pandas_ta as an alternative or any other

In [14]:
import pandas as pd
import vectorbt as vbt
import numpy as np
import talib
import openpyxl

import the financial data and format it to a standard format.(set the time index, make sure the data type is time).

In [15]:
#get the data
trend= pd.read_csv('EURUSD_M15.csv',sep='\t', engine = 'python')
#set time as index

trend=trend.set_index('Time')
# trend['trend']=np.nan

# fastk_period=14 
# fastd_period=6 
# SLOWMA=960
# FASTMA=8
# atr_period=14 
# Breakeven_no_Xatr=3 
# tp_no_Xatr=3
#get all indicators


assign each column to a variable.
convert each column to numpy array. 
indicatofactory does not handle pandas well.
indicatorfactory does not handle multiple columns well
confirm they are of the same shape

In [3]:
Open =trend['Open'].to_numpy()

High=trend['High'].to_numpy()
Low=trend['Low'].to_numpy()
Close= trend['Close'].to_numpy()
Volume=trend['Volume'].to_numpy()
Volume

array([ 2073,  2169,  1412, ..., 10610,  9936,  6798], dtype=int64)

define a function. develop your strategy in it. the data parameters should come first then the input parameters, second.
use the talib indicator and convert it to numpy. if using any other library make sure it is a numpy array. ...... make sure the return is an array. not a multiple column array but a single column array. all new arrays should be seperated by a comma

In [4]:
def combine_indicators(Open, High, Low, Close, Volume, fastk_period=14,fastd_period=6,SLOWMA=960, FASTMA=8, 
                       atr_period=14,Breakeven_no_Xatr=3,tp_no_Xatr=3):

    STOCHF = vbt.IndicatorFactory.from_talib('STOCHF')
    STOCHD=STOCHF.run(High,Low, Close, fastk_period,fastd_period,0 ).fastd.to_numpy()
    STOCHK=STOCHF.run(High,Low, Close, fastk_period,fastd_period,0 ).fastk.to_numpy()
    ATR = vbt.IndicatorFactory.from_talib('ATR')
    atr =ATR.run(High, Low, Close, atr_period).real.to_numpy()
    MA= vbt.IndicatorFactory.from_talib('MA')
    ma = MA.run(Close, SLOWMA,0).real.to_numpy()

    FA_MA= vbt.IndicatorFactory.from_talib('MA')
    fa_ma = FA_MA.run(Close, FASTMA,0).real.to_numpy()
    #indicator is ready
    
 
    tp=tp_no_Xatr* atr

    


    sl=Breakeven_no_Xatr*atr
    sl2= (abs(sl-Close)/Close)*100
    
    long_entry=np.where((STOCHD<40) & (STOCHD>STOCHK) & (Close> ma),1,0)
    short_entry = np.where((STOCHD>70) & (STOCHD<STOCHK) & (Close <ma),2,0)


    # buy=1
    # sell=2

    #converting to percentage
    tp2=abs(((tp - Close)/ (Close)))*100
    breakeven_Close=np.where((short_entry==2), Close, np.nan)
    breakeven_Close=np.where((short_entry==2), Close, np.nan)
   
    if True:
    
        for i in range(breakeven_Close.size):
            if np.isnan(breakeven_Close[i]):
                breakeven_Close[i]=breakeven_Close[i-1]
        compared_with_longentry=breakeven_Close+(Breakeven_no_Xatr*atr)
        compared_with_shortentry=breakeven_Close-(Breakeven_no_Xatr*atr)
    #     
        breakeven_Close=np.nan_to_num(breakeven_Close, nan=0)
        exits=np.where((((fa_ma>breakeven_Close) & (np.roll(fa_ma,1)<breakeven_Close))>compared_with_longentry),1,0 )


        short_exits=np.where((((fa_ma<breakeven_Close) & (np.roll(fa_ma,1)>breakeven_Close))<compared_with_shortentry),1,0 )
    else:
        exits=np.where(((fa_ma>Close) & (np.roll(fa_ma,1)<Close)),1,0 )
        short_exits=np.where(((fa_ma<Close) & (np.roll(fa_ma,1)>Close)),1,0 )
            
    


    return tp, sl, sl2, long_entry, short_entry, tp2, breakeven_Close, exits, short_exits


indicatorfactory......
the class name can be anything.
the short name can be any shorter version.
the input names= this should be exactly the samee with the data parameter with the function designed above
param names = this is the indicator values of each parameter in your function. it should be exactly thesame in terms of the string name.
output names= this should be exactly the same with the returns of your function
.from_apply_func( this is for setting default values. first you input the function name you developed above, then the default parameters. there are other parameters that can go into this.


In [5]:
ind = vbt.IndicatorFactory(class_name= "combination",
                           short_name="comb",
                           input_names=['Open', 'High', 'Low','Close','Volume'],
                           param_names=['fastk_period', 'fastd_period', 'SLOWMA', 'FASTMA', 'atr_period', 'Breakeven_no_Xatr', 'tp_no_Xatr'],
                           output_names=['tp', 'sl', 'sl2', 'long_entry', 'short_entry', 'tp2', 'breakeven_Close', 'exits', 'short_exits']
                           ).from_apply_func(
                                            combine_indicators,
                                            fastk_period=14, 
                                            fastd_period=6, 
                                            SLOWMA=960,
                                            FASTMA=8,
                                            atr_period=14,
                                            Breakeven_no_Xatr=3,
                                            tp_no_Xatr=3, 
                                            keep_pd = True)

for walkforward analysis the data should be in pandas. recall the columns

In [6]:
Open1 =trend['Open']

High1=trend['High']
Low1=trend['Low']
Close1= trend['Close']
Volume1=trend['Volume']
Volume1

Time
2018-10-23 01:30:00     2073
2018-10-23 01:45:00     2169
2018-10-23 02:00:00     1412
2018-10-23 02:15:00     1717
2018-10-23 02:30:00     1864
                       ...  
2022-10-26 16:45:00     7303
2022-10-26 17:00:00    11280
2022-10-26 17:15:00    10610
2022-10-26 17:30:00     9936
2022-10-26 17:45:00     6798
Name: Volume, Length: 100000, dtype: int64

create two tupples, one for the in sample the other for the outsample. then each is divided into two the date and the prices.
the reason for not using the whole dataframe is because indicatorfactory input prefers it column by colum.
rolling_split is the name of the function. there is a function similar called range_split(n=, range, len) it has two outputs the rangedf and the rand indexes seperated by a comma. back to rolling_split. the n is for numple of splits, window_len= the amount of each split, set_lens=this is for the outsample you don't after fill after the comma that is for validity, if you fill this there would be need to make the output tupple 3. left_to_right should be false except you want to change the order of the output

In [7]:
(in_sample_pricesOpen1, in_sample_datesOpen1), (out_sample_pricesOpen1, out_sample_datesOpen1) = Open1.vbt.rolling_split(n=20, window_len=8000, set_lens=(2400,), left_to_right=False)
(in_sample_pricesHigh1, in_sample_datesHigh1), (out_sample_pricesHigh1, out_sample_datesHigh1) = High1.vbt.rolling_split(n=20, window_len=8000, set_lens=(2400,), left_to_right=False)
(in_sample_pricesLow1, in_sample_datesLow1), (out_sample_pricesLow1, out_sample_datesLow1) = Low1.vbt.rolling_split(n=20, window_len=8000, set_lens=(2400,), left_to_right=False)
(in_sample_pricesClose1, in_sample_datesClose1), (out_sample_pricesClose1, out_sample_datesClose1) = Close1.vbt.rolling_split(n=20, window_len=8000, set_lens=(2400,), left_to_right=False)
(in_sample_pricesVolume1, in_sample_datesVolume1), (out_sample_pricesVolume1, out_sample_datesVolume1) = Volume1.vbt.rolling_split(n=20, window_len=8000, set_lens=(2400,), left_to_right=False)

# (in_sample_prices, in_sample_dates), (out_sample_prices, out_sample_dates) = trend.vbt.rolling_split(n=20, window_len=360, set_lens=(108,), left_to_right=False)

ind.run is to run the indicator factor take note that any parameter not inputted the default would be used'
my_list is used to collect the performance parameters
the for loop is to loop through each of the dataset produced by the split.
the parameters for the portfolio.from_signals  are then collected there are other alternatives if you read the documentation.
i like using pandas for the portfolio.from_signal price so that the output can have the dates with them. it doesn't seem to complain.
the freq is the same that of pandas time freqencies= W,M,both is for minute(T,min),D,S.
the porfolio has a lot functions in the class. i simply called for the total profit for the optimization it can be changed.

In [8]:
my_list= []
for i in range(in_sample_pricesClose1.shape[1]):
    
    res=ind.run(in_sample_pricesOpen1[i].to_numpy(), in_sample_pricesHigh1[i].to_numpy(), in_sample_pricesLow1[i].to_numpy(), in_sample_pricesClose1[i].to_numpy(),in_sample_pricesVolume1[i].to_numpy(),[14,38,60],[14,38,60])
    entries=res.long_entry==1
    exits=res.exits
    short_entries = res.short_entry==1
    short_exits=res.short_exits
    exits= res.short_exits
    sl_stop=res.sl2
    tp_stop=res.tp2
    pf = vbt.Portfolio.from_signals(close=trend["Close"].loc[in_sample_datesClose1[i]], entries= entries, exits = exits, short_entries=short_entries,short_exits= short_exits,sl_stop=sl_stop, tp_stop=tp_stop, freq='15T')
    performance = pf.total_profit()
#     heatmap = performance.vbt.heatmap(
#     x_labels='comb_fastd_period',
#     y_labels='comb_fastk_period',)
#     my_list.append([performance.idxmax(), performance.max()])
#     print(portfolio.total_profit())
my_list

[[(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0],
 [(14, 14), 0.0]]

In [9]:
# my_list[0][0][0]
# my_list[i][0][1]

sample as above but for the outsample. remember only the parameters with the most profit is tested on the outsample

In [10]:
my_list_out_sample= []
my_list
for i in range(out_sample_pricesClose1.shape[1]):
    
    res=ind.run(out_sample_pricesOpen1[i].to_numpy(), out_sample_pricesHigh1[i].to_numpy(), out_sample_pricesLow1[i].to_numpy(), out_sample_pricesClose1[i].to_numpy(),out_sample_pricesVolume1[i].to_numpy(),my_list[i][0][0],my_list[i][0][1])
    entries=res.long_entry==1
    exits=res.exits
    short_entries = res.short_entry==1
    short_exits=res.short_exits
    exits= res.short_exits
    sl_stop=res.sl2
    tp_stop=res.tp2
    pf= vbt.Portfolio.from_signals(close=trend["Close"].loc[out_sample_datesClose1[i]], entries= entries, exits = exits, short_entries=short_entries,short_exits= short_exits,sl_stop=sl_stop, tp_stop=tp_stop, freq='15T')
    performance = pf.total_profit()
    my_list_out_sample.append(performance)
#     print(portfolio.total_profit())
my_list_out_sample

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

after you have got the parameters you need. use the codes below to have the data to judge live trades.

In [ ]:


RP=pd.DataFrame(columns =['id','col','size','entry_idx','entry_price','entry_fees','exit_idx',
                          'exit_price','exit_fees','pnl','return','direction','status','parent_id',
                          'annualized_profit','no_of_trades_per_year',
                          'per_winning_trade','largest_win','average_win','duration_of_average_win','largest_loss',
                          'average_loss','duration_of_average_loss','average_wining_run','largest_number_of_wining_run',
                          'largest_number_of_losing_run','average_losing_run','maximum_drawdown'
                         'duration_of_maximum_drawdown'])
my_list_out_sample= []
my_list
for i in range(out_sample_pricesClose1.shape[1]):
    
    res=ind.run(out_sample_pricesOpen1[i].to_numpy(), out_sample_pricesHigh1[i].to_numpy(), out_sample_pricesLow1[i].to_numpy(), out_sample_pricesClose1[i].to_numpy(),out_sample_pricesVolume1[i].to_numpy(),my_list[i][0][0],my_list[i][0][1])
    entries=res.long_entry==1
    exits=res.exits
    short_entries = res.short_entry==1
    short_exits=res.short_exits
    exits= res.short_exits
    sl_stop=res.sl2
    tp_stop=res.tp2
    pf= vbt.Portfolio.from_signals(close=trend["Close"].loc[out_sample_datesClose1[i]], entries= entries, exits = exits, short_entries=short_entries,short_exits= short_exits,sl_stop=sl_stop, tp_stop=tp_stop, freq='15T')
    performance = pf.total_profit()
    my_list_out_sample.append(performance)

    #1 annualized profit
    annualized_profit = pf.annualized_return()
    #2 number of trades per year
    trades = pf.trades.records_arr
    trades_df=pd.DataFrame(trades, columns=['id','col','size','entry_idx','entry_price','entry_fees','exit_idx','exit_price','exit_fees','pnl','return','direction','status','parent_id'])
    no_of_trades_per_year=pf.trades.open.count()
    #buy=0, sell=1
    #3 percentage of winning trades
    per_winning_trade=pf.trades.win_rate()
    #4 largest win
    largest_win=pf.trades.returns.max()
    # 6 average win
    average_win=pf.trades.winning.returns.mean()
    #7 length of average win
    duration_of_average_win=pf.trades.winning.duration.mean()
    #8 largest loss
    largest_loss=pf.trades.returns.min()
    #10 average loss
    average_loss=pf.trades.losing.returns.mean()
    #11 length of time in average loss
    duration_of_average_loss=pf.trades.losing.duration.mean()
    #12 average wining run
    trades_df['pnl_int']=np.nan
    trades_df.loc[trades_df['pnl']<0,'pnl_int']=-1
    trades_df.loc[trades_df['pnl']>0,'pnl_int']=1
    trades_df['pnl_int1']=trades_df['pnl_int'][trades_df['pnl_int']>0]
    trades_df['pnl_int-1']=trades_df['pnl_int'][trades_df['pnl_int']<0]
    consecutive = trades_df['pnl_int1']!=trades_df['pnl_int1'].shift(1)
    counts=consecutive.cumsum()
    trades_df['counts']= counts
    group_counts= trades_df['counts'].groupby(counts).size()
    group_counts[group_counts==1] = 0
    group_counts.max()
    #average wining run
    average_wining_run=group_counts[group_counts>1].mean()
    #largest winning run
    largest_number_of_wining_run=group_counts.max()
    #13 average losing run
    consecutives = trades_df['pnl_int-1']!=trades_df['pnl_int-1'].shift(1)
    countlos=consecutives.cumsum()
    trades_df['countlos']= countlos
    group_countlos= trades_df['countlos'].groupby(countlos).size()
    group_countlos[group_countlos==1] = 0
    #largest losing run
    largest_number_of_losing_run=group_countlos.max()
    #average  losing run
    average_losing_run=group_countlos[group_countlos>1].mean()
    #maximum drawdown
    maximum_drawdown=pf.drawdowns.max_drawdown()
    #duration of maximum drawdown
    duration_of_maximum_drawdown=pf.drawdowns.max_duration()

    RP1=RP['id','col','size','entry_idx','entry_price','entry_fees','exit_idx','exit_price','exit_fees','pnl','return','direction','status','parent_id'].append(trade_df)
    RP=RP.append([{'annualized_profit':annualized_profit}])
    RP=RP.append([{'no_of_trades_per_year':no_of_trades_per_year}])
    RP=RP.append([{'per_winning_trade':per_winning_trade}])
    RP=RP.append([{'largest_win':largest_win}])
    RP=RP.append([{'average_win':average_win}])
    RP=RP.append([{'duration_of_average_win':duration_of_average_win}])
    RP=RP.append([{'largest_loss':largest_loss}])
    RP=RP.append([{'average_loss':average_loss}])
    RP=RP.append([{'duration_of_average_loss':duration_of_average_loss}])
    RP=RP.append([{'average_wining_run':average_wining_run}])
    RP=RP.append([{'largest_number_of_wining_run':largest_number_of_wining_run}])
    RP=RP.append([{'largest_number_of_losing_run':largest_number_of_losing_run}])
    RP=RP.append([{'average_losing_run':average_losing_run}])
    RP=RP.append([{'maximum_drawdown':maximum_drawdown}])
    RP=RP.append([{'duration_of_maximum_drawdown':duration_of_maximum_drawdown}])
    RP=RP.append(RP1)
#     print(portfolio.total_profit())
my_list_out_sample

In [11]:
# #   store split data in tuples
# (in_sample_prices, in_sample_dates), (out_sample_prices, out_sample_dates) = price.vbt.rolling_split(n=20, window_len=360, set_lens=(108,), left_to_right=False)

# print(in_sample_dates)
# print(out_sample_dates)
# print(out_sample_prices[19])

# run combinations on in sample dates, sort by sharpe ratio
# windows = numpy.arange(10, 50)
# fast_ma, slow_ma = vbt.MA.run_combs(in_sample_prices, windows)
# entries = fast_ma.ma_crossed_above(slow_ma)
# exits = fast_ma.ma_crossed_below(slow_ma)
# portfolio = vbt.Portfolio.from_signals(in_sample_prices, entries, exits, freq='1d', direction='both')
# performance = portfolio.sharpe_ratio()
# print(performance[performance.groupby('split_idx').idxmax()].index)

In [12]:
#1 annualized profit
pf.annualized_return()

0.0

In [13]:

#buy=0, sell=1

trades = pf.trades.records_arr
trades_df=pd.DataFrame(trades, columns=['id','col','size','entry_idx','entry_price','entry_fees','exit_idx','exit_price','exit_fees','pnl','return','direction','status','parent_id'])
trades_df


,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id


In [ ]:

#2 number of trades per year
pf.trades.open.count()

In [ ]:

#3 percentage of winning trades
pf.trades.win_rate()

In [ ]:
#4 largest win
pf.trades.returns.max()

In [ ]:
# length of largest win
#entry_idx-exit_idx
trades_df[trades_df['return']==trades_df['return'].max()]['exit_idx']-trades_df[trades_df['return']==trades_df['return'].max()]['entry_idx']

In [ ]:
# 6 average win
pf.trades.winning.returns.mean()

In [ ]:
#7 length of average win
pf.trades.winning.duration.mean()

In [ ]:
#8 largest loss
pf.trades.returns.min()

In [ ]:
#9 length of time in largest loss
trades_df[trades_df['return']==trades_df['return'].min()]['exit_idx']-trades_df[trades_df['return']==trades_df['return'].min()]['entry_idx']


In [ ]:

#10 average loss
pf.trades.losing.returns.mean()

In [ ]:

#10 average loss
pf.trades.losing.returns.mean()

In [ ]:
#11 length of time in average loss
pf.trades.losing.duration.mean()

In [ ]:
#12 average wining run
trades_df['pnl_int']=np.nan
trades_df.loc[trades_df['pnl']<0,'pnl_int']=-1
trades_df.loc[trades_df['pnl']>0,'pnl_int']=1
trades_df['pnl_int1']=trades_df['pnl_int'][trades_df['pnl_int']>0]
trades_df['pnl_int-1']=trades_df['pnl_int'][trades_df['pnl_int']<0]
consecutive = trades_df['pnl_int1']!=trades_df['pnl_int1'].shift(1)
counts=consecutive.cumsum()
trades_df['counts']= counts
group_counts= trades_df['counts'].groupby(counts).size()
group_counts[group_counts==1] = 0
group_counts.max()

In [ ]:
#average wining run
group_counts[group_counts>1].mean()

In [ ]:
#largest winning run
group_counts.max()

In [ ]:
#13 average losing run
consecutives = trades_df['pnl_int-1']!=trades_df['pnl_int-1'].shift(1)
countlos=consecutives.cumsum()
trades_df['countlos']= countlos
group_countlos= trades_df['countlos'].groupby(countlos).size()
group_countlos[group_countlos==1] = 0

In [ ]:

#largest losing run
group_countlos.max()

In [ ]:
#average  losing run
group_countlos[group_countlos>1].mean()

In [ ]:
#maximum drawdown
pf.drawdowns.max_drawdown()

In [ ]:
#duration of maximum drawdown
pf.drawdowns.max_duration()